# But du notebook

Ce notebook explique à quoi sert le padding et qu'est-ce qu'un "attention mask".

# Importation des modules

In [11]:
from transformers import AutoTokenizer
import torch

# Padding

In [10]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sentences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this."
]

tokens = [tokenizer.tokenize(sentence) for sentence in sentences]
ids = [tokenizer.convert_tokens_to_ids(token) for token in tokens]

print(ids[0])
print(ids[1])

[146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119]
[146, 4819, 1142, 119]


Les deux phrases ci-dessus n'ont pas la même longueur. Si on essaye de créer un tensor à partir des listes d'IDs générés, on va avoir une erreur car un tensor doit être rectangulaire.

In [12]:
inputs_ids = torch.tensor(ids)

ValueError: expected sequence of length 16 at dim 1 (got 4)

Pour pallier ça, on ajoute des tokens de padding pour que chaque phrase fasse la même longueur. Pour savoir quel token de padding ajouter, on peut faire `tokenizer.pad_token_id`.

In [19]:
tokenizer.pad_token_id

0

In [18]:
# Ajout de 0 pour le padding
ids = [
    [146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119],
    [146, 4819, 1142, 119,    0,    0,   0,     0,     0,    0,    0,    0,    0,    0,    0,   0]
]

# Création des tensors
inputs_ids = torch.tensor(ids)

# Attention mask

Les attention layers prennnet en compte tous les tokens de la phrase. Pour être sûr d'obtenir le même résultat avec ou sans les tokens de padding, il faut préciser aux attention layers qu'il ne faut pas faire attention aux tokens de padding.

Pour ça, on crée un "attention mask". C'est un tensor qui fait la même taille que `input_ids` dans lequel : 

- `0` = ne pas faire attention
- `1` = faire attention.

In [ ]:
attention_mask = torch.tensor(
    [
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ]
)

# All-in-one

Au final, ces deux étapes sont réalisées quand on précise `padding = True` : 

In [22]:
tokenizer(sentences, padding=True)

{'input_ids': [[101, 146, 112, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119, 102], [101, 146, 4819, 1142, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}